In [2]:
import pandas as pd

In [3]:
ekb = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\EKB_places_with_description.csv')
nn = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\NN_places_with_description.csv')
vlad = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\Vladimir_places_with_description.csv')
yar = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\Yaroslavl_places_with_description.csv')

In [4]:
ekb = ekb.drop_duplicates(subset='Name', keep='first')
nn = nn.drop_duplicates(subset='Name', keep='first')
vlad = vlad.drop_duplicates(subset='Name', keep='first')
yar = yar.drop_duplicates(subset='Name', keep='first')

In [5]:
def make_global_indexes(prefix, df_places):
    df_places = df_places.drop_duplicates(subset=['Name'])
    df_places = df_places.reset_index(drop=True)
    df_places['global_id'] = prefix + '_' + df_places.index.astype(str)
    return df_places

def make_local_index(prefix, df_images):
    df_images = df_images.drop_duplicates(subset=['image'])
    df_images = df_images.reset_index(drop=True)
    df_images['local_id'] = prefix + '_' + df_images.index.astype(str)    
    return df_images

In [14]:
ekb_places

,XID,Name,Kind,City,OSM,WikiData,Rate,Lon,Lat,description,global_id
0,W38411380,Динамо,"sport,architecture,historic_architecture,inter...",Екатеринбург,way/38411380,Q37996725,3h,60.600349,56.845398,спорткомплекс в Екатеринбурге,ekb_0
1,N2885181131,№32 Дом обороны,"architecture,historic_architecture,interesting...",Екатеринбург,node/2885181131,Q55209768,3h,60.601315,56.834167,здание в Екатеринбурге,ekb_1
2,W38581890,Дом обороны,"architecture,historic_architecture,interesting...",Екатеринбург,way/38581890,Q55209768,3h,60.602409,56.835133,здание в Екатеринбурге,ekb_2
3,N1930476141,№28 Здание городской электростанции «Луч»,"architecture,historic_architecture,interesting...",Екатеринбург,node/1930476141,Q55154121,3h,60.607430,56.833691,NaN,ekb_3
4,N3002726097,№27 Дом Г.Н. Скрябина,"architecture,historic_architecture,interesting...",Екатеринбург,node/3002726097,Q55232375,3h,60.607075,56.834225,NaN,ekb_4
...,...,...,...,...,...,...,...,...,...,...,...
92,Q4411412,Свято-Никольский храм (Екатеринбург),"religion,churches,interesting_places,other_chu...",Екатеринбург,NaN,Q4411412,3h,60.595821,56.826569,православный храм в Екатеринбурге,ekb_92
93,Q4504106,Центральная городская библиотека им. А. И. Гер...,"historic_architecture,architecture,interesting...",Екатеринбург,NaN,Q4504106,3h,60.612167,56.821865,Библиотека в Екатеринбурге,ekb_93
94,Q28381390,Дом Гайдара,"historic_architecture,architecture,interesting...",Екатеринбург,NaN,Q28381390,3h,60.591591,56.845341,NaN,ekb_94
95,Q50359307,Дом дворянки Е. О. Селивановой,"other,unclassified_objects,interesting_places,...",Екатеринбург,NaN,Q50359307,3h,60.609085,56.840805,NaN,ekb_95


In [6]:
ekb_places = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\EKB_places_with_description.csv')
ekb_images = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\EKB_images.csv')

nn_places = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\NN_places_with_description.csv')
nn_images = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\NN_images.csv')

vladimir_places = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\Vladimir_places_with_description.csv')
vladimir_images = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\Vladimir_images.csv')

yaroslavl_places = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\Yaroslavl_places_with_description.csv')
yaroslavl_images = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\Yaroslavl_images.csv')

yaroslavl_images = yaroslavl_images.rename({'img': 'image'}, axis=1)

def make_global_indexes(prefix, df_places):
    df_places = df_places.drop_duplicates(subset=['Name'])
    df_places = df_places.reset_index(drop=True)
    df_places['global_id'] = prefix + '_' + df_places.index.astype(str)
    return df_places

def make_local_index(prefix, df_images):
    df_images = df_images.drop_duplicates(subset=['image'])
    df_images = df_images.reset_index(drop=True)
    df_images['local_id'] = prefix + '_' + df_images.index.astype(str)    
    return df_images

# ekb
ekb_places = make_global_indexes('ekb', ekb_places)
ekb_images = make_local_index('ekb_local', ekb_images)
full_ekb = pd.merge(ekb_places, ekb_images, left_on='Name', right_on='name')[['Name', 'Kind', 'City', 'Lon', 'Lat', 'global_id', 'image', 'local_id', 'Rate', 'description']]
# nn
nn_places = make_global_indexes('nn', nn_places)
nn_images = make_local_index('nn_local', nn_images)
full_nn = pd.merge(nn_places, nn_images, left_on='Name', right_on='name')[['Name', 'Kind', 'City', 'Lon', 'Lat', 'global_id', 'image', 'local_id', 'Rate', 'description']]
# vladimir
vladimir_places = make_global_indexes('vlad', vladimir_places)
vladimir_images = make_local_index('v_local', vladimir_images)
full_vladimir = pd.merge(vladimir_places, vladimir_images, left_on='Name', right_on='name')[['Name', 'Kind', 'City', 'Lon', 'Lat', 'global_id', 'image', 'local_id', 'Rate', 'description']]
# yaroslavl
yaroslavl_places = make_global_indexes('y', yaroslavl_places)
yaroslavl_images = make_local_index('y_local', yaroslavl_images)
full_yaroslavl = pd.merge(yaroslavl_places, yaroslavl_images, left_on='Name', right_on='name')[['Name', 'Kind', 'City', 'Lon', 'Lat', 'global_id', 'image', 'local_id', 'Rate', 'description']]


In [7]:
import joblib

ekb_bad_ids = [6, 18, 20, 21, 23, 24, 25, 27, 31, 40, 63, 67, 70, 114, 117, 122, 123, 124, 125, 132, 136, 139, 142, 150, 151, 155, 159, 160, 164, 166, 168, 169, 170, 171, 173, 176,
           179, 180, 182, 184, 192, 195, 199, 200, 203, 207, 208, 210, 219, 220, 221, 217, 256, 267, 288, 292, 294, 309, 319, 321, 322, 324, 325, 326, 330, 331, 332, 333, 336, 337, 339, 340, 341, 342,
           343, 344, 350, 352, 356, 365, 366, 374, 377, 379, 380, 383, 386, 387, 388, 393, 394, 395, 396, 400, 404, 405, 406, 408, 409, 411, 412, 413, 414, 415, 416, 418, 419, 421, 424,
           425, 426, 427, 429, 477, 482, 487, 517, 525, 545, 547, 574, 593, 597, 599, 604, 606, 609, 610, 612, 613, 615, 617, 627, 628, 632, 634, 635, 636, 639, 654, 657, 666, 687, 688, 695,
           698, 715, 716, 722, 725, 726, 779, 784, 787, 802, 811, 813, 815, 817, 819, 821, 834, 826, 837, 838, 839, 840, 842, 843, 844, 845, 846, 847, 848, 850, 851, 852, 853, 854, 855, 856,
           857, 858, 860, 884, 905, 910, 911, 918, 919, 921, 922, 925, 926, 927, 928, 929, 932, 933, 934, 935, 936, 938, 939, 982, 984, 985, 993, 998, 1005, 1011, 1066, 1067, 1082, 1086,
           1087, 1090, 1091, 1092, 1095, 1096, 1099, 1101, 1103, 1108, 1110, 1111, 1112, 1113, 1116, 1117, 1119, 1132, 1141, 1239, 1252, 1276, 1277, 1278, 1279, 1280, 1290, 1352, 1364,
           1383, 1392, 1398, 1402, 1417, 1421, 1465, 1472, 1475, 1487, 1489, 1492, 1496, 1497, 1547, 1552, 1554, 1555, 1556, 1557, 1559, 1560, 1562, 1563, 1564, 1567, 1584, 1586, 1587,
           1559, 1590, 1592, 1593, 1595, 1596, 1597, 1598, 1601, 1602, 1605, 1616, 1625, 1626, 1628, 1629, 1631, 1636, 1638, 1643, 1682, 1713, 1750, 1753, 1766, 1773,
           1778, 1788, 1799, 1802, 1807, 1808, 1809, 1810, 1816, 1817, 1823, 1824, 1827, 1829, 1832, 1834, 1835, 1843, 1844, 1890, 1898, 1918, 1921, 1946, 1947, 2002, 2003,
           2008, 2026, 2035, 2081, 2082, 2090, 2094, 2098, 2100, 2111, 2119, 2132, 2136,2142,2145,2149,2153,2156,2161,2162,2163,2175,2212,2223,2230,
2253,2263,2267,2271,2289,2324,2331,2332,2338,2340,2341,2342,2343,2345,2346,2347,2348,2349,2350,2351,2352,2353,2354,2355,2356,2357,2358,2359,
2366,2376,2383,2408,2414,2416,2422,2509,2510,2584,2587,2598,2603,2604,2605,2660,2666, 2674,2695,2704,2705,2706,2707,2708,2709,2710,2711,2712,2713,2714,
2716,2717,2718,2719,2720,2778,2802,2809,2813,2822,2823,2824,2859,2860,2867,2872,2873,2874,2875,2786,2877,2878,2879,2931,2939,2961]

nn_bad_ids = joblib.load(r'D:\ai-sightseeing-nto\backend\data\bad_ids_nn.pkl')

yaroslavl_bad_ids = joblib.load(r'D:\ai-sightseeing-nto\backend\data\bad_yaroslavl.pkl')

vladimir_bad_ids = joblib.load(r'D:\ai-sightseeing-nto\backend\data\vladimir_trash_list.pkl')

ekb = full_ekb.copy()
ekb = ekb.drop(index=ekb_bad_ids)
ekb = ekb.reset_index(drop=True)

nn = full_nn.copy()
nn = nn.drop(index=nn_bad_ids)
nn = nn.reset_index(drop=True)

yar = full_yaroslavl.copy()
yar = yar.drop(index=yaroslavl_bad_ids)
yar = yar.reset_index(drop=True)

vlad = full_vladimir.copy()
vlad = vlad.drop(index=vladimir_bad_ids)
vlad = vlad.reset_index(drop=True)


In [8]:


names_to_change_yaroslavl = {'Н.А. Некрасову':'памятник Н.А. Некрасову', 'Николая Чудотворца':'церковь Николая Чудотворца',
'Дом муз':'дом Бибикова', 'М.А. Богданович': 'памятник М.А. Богдановичу', 'М.В. Фрунзе':'памятник М.В. Фрунзе', 
'Ф. Г. Волкову':'памятник Ф. Г. Волкову', 'Фёдор Иванович Толбухин':'памятник Ф.И. Толбухину', 
'Г.М. Димитрову':'памятник Г.М. Димитрову'
}
names_to_change_ekb = {'Динамо':'стадион Динамо', 'Литературная жизнь Урала XX века':'музей "Литературная жизнь Урала XX века"',
                       'Серго Орджоникидзе':'памятник Серго Орджоникидзе', 'Н. И. Кузнецову':'памятник Н. И. Кузнецову',
                       'Малышеву': 'памятник Малышеву', 'Комсомолу Урала':'памятник Комсомолу Урала', 
                       'Ленин': 'памятник Ленину'}
names_to_change_nn = {'Скоба':'ресторан Скоба', 'Рекорд':'центр культуры Рекорд', 'Локомотив':'стадион Локомотив',
                      'М. Горькому':'памятник М. Горькому'}
names_to_change_vladimir = {'Кресень':'фольклорный театр Кресень', 'Дружба':'парк Дружба', 'Старый Владимир':'музей Старый Владимир',
                           'Огни Владимира':'музей Огни Владимира', 'Старая аптека': 'музей Старая аптека', 'Бобёр':'скульптура Бобёр',
                           'Владимирская вишня': 'скульптура Владимирская вишня', 'Такса':'скульптура Такса', 
                           'Мальчик с рогаткой':'скульптура Мальчик с рогаткой', 'Фавор':'театр Фавор', 'Филёр':'скульптура Филёр',
                           'Художник на пленэре':'скульптура Художник на пленэре', 'Аптекарь':'скульптура Аптекарь'}

ekb['Name'] = ekb['Name'].replace(names_to_change_ekb)
nn['Name'] = nn['Name'].replace(names_to_change_nn)
yar['Name'] = yar['Name'].replace(names_to_change_yaroslavl)
vlad['Name'] = vlad['Name'].replace(names_to_change_vladimir)

full_df = pd.concat([ekb, nn, yar, vlad])
full_df = full_df.reset_index(drop=True)


In [11]:
ekb.to_csv(r'D:\ai-sightseeing-nto\backend\app\models\text2img\weights\ekb.csv')
yar.to_csv(r'D:\ai-sightseeing-nto\backend\app\models\text2img\weights\yar.csv')
nn.to_csv(r'D:\ai-sightseeing-nto\backend\app\models\text2img\weights\nn.csv')
vlad.to_csv(r'D:\ai-sightseeing-nto\backend\app\models\text2img\weights\vlad.csv')

In [12]:
vlad

,Name,Kind,City,Lon,Lat,global_id,image,local_id,Rate,description
0,Церковь Никиты Великомученика,"religion,churches,interesting_places,eastern_o...",Владимир,40.397797,56.130745,vlad_0,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,v_local_0,3h,NaN
1,Церковь Никиты Великомученика,"religion,churches,interesting_places,eastern_o...",Владимир,40.397797,56.130745,vlad_0,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,v_local_1,3h,NaN
2,Церковь Никиты Великомученика,"religion,churches,interesting_places,eastern_o...",Владимир,40.397797,56.130745,vlad_0,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,v_local_2,3h,NaN
3,Церковь Никиты Великомученика,"religion,churches,interesting_places,eastern_o...",Владимир,40.397797,56.130745,vlad_0,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,v_local_3,3h,NaN
4,Церковь Никиты Великомученика,"religion,churches,interesting_places,eastern_o...",Владимир,40.397797,56.130745,vlad_0,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,v_local_4,3h,NaN
...,...,...,...,...,...,...,...,...,...,...
2044,Владимирским железнодорожникам погибшим в ВОВ,"historic,burial_places,interesting_places,war_...",Владимир,40.421490,56.130661,vlad_96,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,v_local_3311,1,NaN
2045,Владимирским железнодорожникам погибшим в ВОВ,"historic,burial_places,interesting_places,war_...",Владимир,40.421490,56.130661,vlad_96,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,v_local_3312,1,NaN
2046,Владимирским железнодорожникам погибшим в ВОВ,"historic,burial_places,interesting_places,war_...",Владимир,40.421490,56.130661,vlad_96,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,v_local_3313,1,NaN
2047,Владимирским железнодорожникам погибшим в ВОВ,"historic,burial_places,interesting_places,war_...",Владимир,40.421490,56.130661,vlad_96,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,v_local_3314,1,NaN


In [61]:
full_df[['global_id', 'local_id']].to_csv(r'D:\ai-sightseeing-nto\backend\app\models\text2img\weights\id_conc.csv', index = False)

In [62]:
full_df['Kind'] = full_df['Kind'].apply(replace_categories)

In [63]:
final = full_df[['Name','Kind','City','Lon','Lat','global_id','Rate','description']].drop_duplicates().rename(columns={'global_id': 'id', 'Name': 'name', 'City':'city', 'Lon':'lon', 'Lat':'lat', 'Rate':'rate', 'Kind': 'type'})

In [65]:
final = full_df[['global_id','image', 'local_id']].drop_duplicates().rename(columns={'global_id': 'id', 'image': 'base64'})

In [66]:
counts = final['base64'].value_counts()
duplicates = counts[counts > 1].index

final = final[~final['base64'].isin(duplicates)]

In [67]:
final

,id,base64,local_id
0,ekb_0,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,ekb_local_0
1,ekb_0,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,ekb_local_1
2,ekb_0,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,ekb_local_2
3,ekb_0,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,ekb_local_3
4,ekb_0,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,ekb_local_4
...,...,...,...
10371,vlad_96,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,v_local_3311
10372,vlad_96,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,v_local_3312
10373,vlad_96,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,v_local_3313
10374,vlad_96,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,v_local_3314


In [50]:
final['base64'].unique().shape

(13276,)

In [64]:
final.to_csv(r'D:\ai-sightseeing-nto\backend\app\connection\final_db.csv', index=False)

In [68]:
final.to_csv(r'D:\ai-sightseeing-nto\backend\app\connection\photo_obj_db.csv', index=False)

### object

In [59]:
col = ['XID', 'Name', 'City', 'Lon', 'Lat', 'Rate', 'description', 'Kind']
final = pd.concat([ekb[col], nn[col], vlad[col], yar[col]])

In [60]:
final = final.rename(columns={'XID': 'id', 'Name': 'name', 'City':'city', 'Lon':'lon', 'Lat':'lat', 'Rate':'rate', 'Kind': 'type'})

In [67]:
final['rate'].unique()

array(['3h', '3', '2', '1', '2h'], dtype=object)

In [25]:
final.to_csv(r'D:\ai-sightseeing-nto\backend\data\final_db.csv')

In [43]:
final.to_csv(r'D:\ai-sightseeing-nto\backend\app\connection\photo_obj_db.csv', index=False)

### kind

In [17]:
category_groups = {
    'Музеи и памятники': ['historic', 'military_museums', 'national_museums', 'biographical_museums', 'war_memorials', 'museums', 'history_museums', 'historic_settlements', 'archaeological_museums', 'historic_house_museums', 'science_museums', 'museums_of_science_and_technology', 'art_galleries', 'local_museums', 'other_technology_museums', 'children_museums', 'zoos', 'open_air_museums', 'installation'],
    'Религиозные места': ['catholic_churches', 'eastern_orthodox_churches', 'churches', 'other_churches', 'cathedrals', 'mosques', 'other_temples'],
    'Рестораны и еда': ['restaurants', 'foods'],
    'Архитектурные объекты': ['historic_architecture', 'other_buildings_and_structures', 'castles', 'bridges', 'towers', 'fortified_towers', 'water_towers', 'fortifications', 'defensive_walls', 'historic_districts', 'other_towers', 'urban_environment','fountains', 'circuses', 'triumpal_archs', 'other_archaeological_sites'],
    'Природные объекты': ['natural_monuments', 'mountain_peaks', 'geological_formations', 'rock_formations', 'nature_reserves', 'gardens_and_parks', 'zoos'],
    'Спорт и развлечения': ['sport', 'concert_halls', 'cinemas', 'opera_houses', 'puppetries', 'stadiums', 'theatres_and_entertainments', 'circuses'],
    'Финансовые и банковские учреждения': ['banks', 'bank'],
    'Туристические объекты': ['tourist_facilities', 'tourist_object', 'railway_stations','squares'],
    'Другие места': ['other', 'unclassified_objects', 'view_points', 'cemeteries']
}


In [51]:
col = ['Kind']
final = pd.concat([ekb[col], nn[col], vlad[col], yar[col]])

In [18]:
def replace_categories(string):
    categories_list = string.split(',')
    keys = []
    for category in categories_list:
        for key, value in category_groups.items():
            if category.strip() in value:
                keys.append(key)
                break
    return ', '.join(keys)

# final['type'] = final['type']

In [105]:
all_cat = []

for i in range(len(final)):
    all_cat.extend(final.iloc[i]['Kind'].split(','))

all_cat_set = set(all_cat)


In [106]:
kind_list = sorted(list(all_cat_set))
id_list = range(1, len(kind_list) + 1)

df = pd.DataFrame({'id': id_list, 'type': kind_list})

In [34]:
df.to_csv(r'D:\ai-sightseeing-nto\backend\app\connection\kind_db.csv', index=False)

### kind obj

In [107]:
df

,id,type
0,1,accomodations
1,2,archaeological_museums
2,3,archaeology
3,4,architecture
4,5,art_galleries
...,...,...
90,91,urban_environment
91,92,view_points
92,93,war_memorials
93,94,water_towers


In [113]:
col = ['Kind', 'XID']
final = pd.concat([ekb[col], nn[col], vlad[col], yar[col]])

In [114]:
final

,Kind,XID
0,"sport,architecture,historic_architecture,inter...",W38411380
1,"architecture,historic_architecture,interesting...",N2885181131
2,"architecture,historic_architecture,interesting...",W38581890
3,"architecture,historic_architecture,interesting...",N1930476141
4,"architecture,historic_architecture,interesting...",N3002726097
...,...,...
95,"historic,archaeology,interesting_places,other_...",Q97161882
96,"historic_architecture,architecture,interesting...",Q4091101
97,"museums,cultural,science_museums,children_muse...",U3739
98,"biographical_museums,museums,cultural,interest...",U3885


In [118]:
df_exploded = final.assign(Kind=final['Kind'].str.split(',')).explode('Kind').rename(columns={'Kind': 'type'})

In [119]:
df_exploded

,type,XID
0,sport,W38411380
0,architecture,W38411380
0,historic_architecture,W38411380
0,interesting_places,W38411380
0,stadiums,W38411380
...,...,...
99,museums,U4106
99,cultural,U4106
99,interesting_places,U4106
99,local_museums,U4106


In [122]:
last = pd.merge(df_exploded, df, on = 'type')

In [123]:
df_id_xid = pd.DataFrame({'id': range(len(last)), 'id_obj': last['XID'], 'id_kind':last['id']})

In [124]:
df_id_xid.to_csv(r'D:\ai-sightseeing-nto\backend\app\connection\obj_kind_db.csv', index=False)

In [125]:
df_id_xid

,id,id_obj,id_kind
0,0,W38411380,82
1,1,W587989335,82
2,2,R2767108,82
3,3,W38411380,4
4,4,N2885181131,4
...,...,...,...
1798,1798,W222984781,46
1799,1799,Q2494278,3
1800,1800,Q97161882,3
1801,1801,Q2494278,62


### photo

In [ ]:
col = ['Kind']
final = pd.concat([ekb[col], nn[col], vlad[col], yar[col]])

In [20]:
ekbph = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\EKB_images.csv')
nnph = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\NN_images.csv')
vladph = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\Vladimir_images.csv')
yarph = pd.read_csv(r'D:\ai-sightseeing-nto\backend\data\Yaroslavl_images.csv')

In [22]:
ekbph = ekbph.drop_duplicates(subset = ['image'])
nnph = nnph.drop_duplicates(subset = ['image'])
vladph = vladph.drop_duplicates(subset = ['image'])
yarph = yarph.drop_duplicates(subset = ['img'])

ekb = ekb.drop_duplicates(subset = ['Name'])
nn = nn.drop_duplicates(subset = ['Name'])
vlad = vlad.drop_duplicates(subset = ['Name'])
yar = yar.drop_duplicates(subset = ['Name'])

yar.rename(columns={'img': 'image'}, inplace=True)


In [23]:
ekb.rename(columns={'Name': 'name'}, inplace=True)
ekbph = pd.merge(ekb, ekbph, on = 'name')

nn.rename(columns={'Name': 'name'}, inplace=True)
nnph = pd.merge(nn, nnph, on = 'name')

vlad.rename(columns={'Name': 'name'}, inplace=True)
vladph = pd.merge(vlad, vladph, on = 'name')

yar.rename(columns={'Name': 'name'}, inplace=True)
yarph = pd.merge(yar, yarph, on = 'name')

In [25]:
yarph.rename(columns={'img': 'image'}, inplace=True)

In [26]:
col = ['XID', 'image']
df = pd.concat([ekbph[col], nnph[col], vladph[col], yarph[col]])

In [27]:
df = df.drop_duplicates()
df = df.rename(columns={'XID':'id_obj', 'image':'base64'})

In [33]:
counts = df['base64'].value_counts()
duplicates = counts[counts > 1].index

df = df[~df['base64'].isin(duplicates)]


In [34]:
df

,id_obj,base64
0,W38411380,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
1,W38411380,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
2,W38411380,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
3,W38411380,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
4,W38411380,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
...,...,...
3958,U4106,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
3959,U4106,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
3960,U4106,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
3961,U4106,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...


In [31]:
df['base64'].unique().shape

(13281,)

In [35]:
df.to_csv(r'D:\ai-sightseeing-nto\backend\app\connection\photo_obj_db.csv', index=False)

In [89]:
df_id_xid = pd.DataFrame({'XID': df['XID'], 'base64': df['image']})
 
df_id_image = pd.DataFrame({'id': range(len(list(set(df['image'].to_list())))), 'base64': list(set(df['image'].to_list()))})

In [91]:
df_id_xid = pd.merge(df_id_xid, df_id_image, on = 'base64')

In [96]:
df_id_xid = df_id_xid[['XID', 'id']].drop_duplicates()

In [97]:
df_id_xid = pd.DataFrame({'id': range(len(df_id_xid)), 'id_obj': df_id_xid['XID'], 'id_photo':df_id_xid['id']})

In [98]:
df_id_xid.to_csv(r'D:\ai-sightseeing-nto\backend\app\connection\obj_photo_db.csv', index=False)

In [86]:
df_id_image.to_csv(r'D:\ai-sightseeing-nto\backend\app\connection\photo_db.csv', index=False)

In [90]:
df_id_image

,id,base64
0,0,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
1,1,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
2,2,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
3,3,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
4,4,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
...,...,...
13276,13276,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
13277,13277,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
13278,13278,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
13279,13279,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
